In [ ]:
import pandas as pd
import json

lucene_lib = "lucene-9.8.0"
tantivy_lib = "tantivy-0.21"
metric_label_microsec="(μs)"

results_location = "../web/build/results.json"

In [ ]:
#compare mean between tantivy and lucene durations
def get_mean_per_query_compare(test_type, library):
    results=json.load(open(results_location, "r"))
    df = results[test_type][library]
    df = pd.json_normalize(df)
    df = df.drop("tags", axis=1)
    df1 = pd.concat([pd.DataFrame(df["duration"].values.tolist())], 
             axis=1, 
             keys=df.columns)
    df1.columns = [f'{i}{j}' for i, j in df1.columns]
    df[f"{library}{metric_label_microsec}"] = df1.mean(axis=1)
    df = df.drop(["duration","count"], axis=1)
    return df

In [ ]:
lucene_df = get_mean_per_query_compare("TOP_100", lucene_lib)
tantivy_df = get_mean_per_query_compare("TOP_100", tantivy_lib)
df = lucene_df.merge(tantivy_df, on="query")
df

In [ ]:
df["diff"] = df[f"{lucene_lib}{metric_label_microsec}"] - df[f"{tantivy_lib}{metric_label_microsec}"]

In [ ]:
df.describe()

In [ ]:
df[df["diff"]<0].describe()

In [ ]:
def get_mean_per_query(test_type, library):
    results=json.load(open(results_location, "r"))
    df = results[test_type][library]
    df = pd.json_normalize(df)
    df = df.drop("tags", axis=1)
    df1 = pd.concat([pd.DataFrame(df["duration"].values.tolist())], 
             axis=1, 
             keys=df.columns)
    df1.columns = [f'{i}{j}' for i, j in df1.columns]
    df[f"meanDuration{metric_label_microsec}"] = df1.mean(axis=1)
    df = df.drop(["duration","count"], axis=1)
    df["library"] = library
    return df


In [ ]:
# started with this, but doesn't work as well as get_mean_per_query_compare for what I was doing
get_mean_per_query("TOP_100", lucene_lib)